In [1]:
from tqdm.auto import tqdm 
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nf
from netCDF4 import Dataset
%matplotlib inline
import glob
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [2]:
datapath="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/Data/ECMWF/ERA5_25kmx3hr/"
path="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/"
output="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/outputs/"
target="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/outputs/targets/"

In [195]:
tracklist = sorted(glob.glob(path+'tracks_nio/*_2001*'))

In [196]:
tracklist

['/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/nio_20010521.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/nio_20011008.csv']

In [197]:
dm1=xr.open_mfdataset([datapath+'/vor/vor_2001.nc',datapath+'/rhum/rhum_2001.nc'])

In [198]:
def largearea_withpres(dataset=None,invar=None,indices=None,tc_irad=None):
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","plev","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices))),
               plev=(["plev"],dataset.plev.data)),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    var_out=np.zeros((len(indices),len(dataset.plev.data),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        for ip in range(len(dm1.plev.data)):
            try:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
            except:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out  

In [199]:
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def smallarea(dataset=None,invar=None,indices=None,tc_irad=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    var_out=np.zeros((len(indices),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out
###############################read year from data and track ####################################

def readyear(year=None):
    dm2 = xr.open_dataset(datapath+'/slev_vars/svars_'+str(year)+'.nc')
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/*_'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

####change year below##########################
dm2,tracklist,era5_date,era5_hour = readyear(2001) 

TCsl_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    ################################################################################################
    
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    
    svarname = ['u10','v10','2mdewtmp','2mtmp','conv_ppt','tot_cld_ice',\
           'tot_cldwtr','tot_cld_rain','vi_div_cld_froz_wtr','vi_div_cld_liq_wtr','vi_div_gpot_flux',\
           'vi_div_ke_flux','vi_div_mass_flux','vi_div_moisture_flux','vi_div_olr_flux','vi_div_tot_enrgy_flux',\
           'vi_ke','vi_pe_inte','vi_pe_ie_latentenrgy','vi_temp','vi_olr','vi_tot_enrgy','vi_moisture_div']
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    ##################################################################################################
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCsl_ts.append(tsdict)

  0%|          | 0/2 [00:00<?, ?it/s]

In [200]:
##########radiation variables############
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def smallarea(dataset=None,invar=None,indices=None,tc_irad=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    var_out=np.zeros((len(indices),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out
###############################read year from data and track ####################################

def readyear(year=None):
    dm2 = xr.open_dataset(datapath+'/slev_vars/radvars_'+str(year)+'.nc')
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/*_'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2001)

TCrl_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    ################################################################################################
    
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    
    svarname = ['cape','inst_10m_wnd_gst','inst_moisture_flux','inst_ssh_flux','surfmean_swr_flux','surfmean_lhf',\
           'surfmean_lwr_flux','surfmean_shf','dwnwrdmean_swr_flux','topmean_lwr_flux','topmean_swr_flux',\
           'vimean_moisture_div','surf_lhf','surf_shf','tot_suprcool_liqwtr','tot_wtr_vpr']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    ##################################################################################################
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCrl_ts.append(tsdict)

  0%|          | 0/2 [00:00<?, ?it/s]

In [201]:
##########precip, mslp, sst variables############ we dont need this anymore.
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def smallarea(dataset=None,invar=None,indices=None,tc_irad=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    var_out=np.zeros((len(indices),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out
###############################read year from data and track ####################################

def readyear(year=None):
    dm2 = xr.open_mfdataset([datapath+'/prate/prates_2001.nc',datapath+'/mslp/mslp_2001.nc',datapath+'/sst/sst_2001.nc'])
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/*_'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2001)

TCpr_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    ################################################################################################
    
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    
    svarname = ['mslp','conv_rrate','ls_rrate','mn_conv_prate','mn_ls_prate','mn_tot_prate','sst']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    ##################################################################################################
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCpr_ts.append(tsdict)

  0%|          | 0/2 [00:00<?, ?it/s]

In [202]:
#For vars with 21 vertical levels vorticity humidity file##################
TCp_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2 
    
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['vort','rhum']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCp_ts.append(ts_pdict)

  0%|          | 0/2 [00:00<?, ?it/s]

In [203]:
dm1=xr.open_dataset(datapath+'/gpot/gpot_2001.nc')

In [204]:
#geopotential
TCgp_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2 
    
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['gpot']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCgp_ts.append(ts_pdict)

  0%|          | 0/2 [00:00<?, ?it/s]

In [205]:
dm1=xr.open_dataset(datapath+'/vvel/w_wnd_2001.nc')

In [206]:
#Vertical velocity
TCvv_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2 
    
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['vvel']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCvv_ts.append(ts_pdict)

  0%|          | 0/2 [00:00<?, ?it/s]

In [207]:
preslv = [str(int(obj)) for obj in dm1.plev.data/100]

In [208]:
dx=xr.open_dataset(datapath+'/tlevs/tlevs_2001.nc')

In [209]:
plev=[str(int(obj)) for obj in dx.plev.data/100]

In [210]:
tracklist

['/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/nio_20010521.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/nio_20011008.csv']

In [211]:
#storms 3
#maindf_as12001=pd.read_csv(target+'2011_w10max_nio_as12001.csv',delimiter=r",")
#maindf_as12001.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
#maindf_as12001=maindf_as12001.drop('a', axis=1)

#extra=pd.read_csv(output+'nio/inder_2011_nio_as12001.csv',delimiter=r",")
#extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
#extra=extra.drop('a', axis=1)

rlev_as12001=pd.DataFrame.from_dict(TCrl_ts[0])
prlev_as12001=pd.DataFrame.from_dict(TCpr_ts[0])
slev_as12001=pd.DataFrame.from_dict(TCsl_ts[0])
prlev_as12001['mslp']=prlev_as12001['mslp']/100

ts1_as12001=pd.concat([slev_as12001,rlev_as12001,prlev_as12001], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[0]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[0]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[0]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[0]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}

plev_as12001=pd.DataFrame.from_dict(alldict)

ts_as12001=pd.concat([ts1_as12001,plev_as12001], axis=1, join='inner')

ts_as12001

,u10,v10,2mdewtmp,2mtmp,conv_ppt,tot_cld_ice,tot_cldwtr,tot_cld_rain,vi_div_cld_froz_wtr,vi_div_cld_liq_wtr,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,2.797020,-1.300709,297.503342,300.435463,0.000992,0.457743,0.077123,0.168191,2.278131e-06,-4.065223e-07,...,-0.531945,-0.591512,-0.569372,-0.393274,-0.236052,-0.162706,-0.144320,-0.100155,-0.047384,-0.020726
1,3.219173,-0.978985,297.747986,300.189552,0.000926,0.449180,0.071876,0.155215,4.057074e-07,-7.695053e-08,...,-0.480585,-0.513593,-0.359564,-0.154222,-0.063871,-0.096027,-0.166329,-0.133193,-0.071666,-0.034648
2,2.672218,-1.660519,297.782349,300.266228,0.001222,0.416765,0.079823,0.116814,-1.804437e-08,-1.251930e-06,...,-0.444614,-0.520131,-0.342196,-0.174109,-0.114000,-0.178794,-0.163676,-0.101833,-0.033161,-0.001448
3,2.963585,-0.259609,298.031876,300.487274,0.001619,0.369974,0.098702,0.112605,2.079643e-06,-6.742775e-07,...,-0.519832,-0.610565,-0.459217,-0.347095,-0.289958,-0.295873,-0.199018,-0.119709,-0.028769,0.012300
4,3.044754,-0.760643,298.218849,300.192093,0.001579,0.489292,0.147114,0.198503,4.214235e-06,-6.970949e-07,...,-0.631488,-0.696444,-0.554210,-0.458196,-0.382395,-0.399617,-0.241413,-0.149184,-0.064776,-0.021619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,1.808327,0.303658,298.515915,300.787300,0.000422,0.043984,0.161987,0.111859,4.921458e-07,4.868489e-07,...,-0.040654,-0.062338,-0.085711,-0.016821,-0.075673,-0.135823,-0.171235,-0.134981,-0.036612,0.021378
63,0.583748,-0.275944,298.645134,301.783989,0.000325,0.069014,0.135358,0.108026,-2.275629e-06,-1.118868e-06,...,-0.057099,-0.047529,-0.053607,-0.081322,-0.178363,-0.170097,-0.194236,-0.130737,-0.046133,-0.002949
64,0.986526,1.515474,298.449669,302.721069,0.000246,0.122222,0.206177,0.187214,1.917942e-06,-8.493662e-07,...,-0.142666,-0.163554,-0.230621,-0.317795,-0.369193,-0.321809,-0.237194,-0.148626,-0.054896,-0.009647
65,1.568237,3.170792,297.796585,304.296021,0.000241,0.107788,0.163889,0.205021,7.151044e-06,7.633353e-07,...,-0.252213,-0.303418,-0.332350,-0.247317,-0.135166,-0.009533,-0.053958,-0.083135,-0.045597,-0.017530


In [212]:
ts_as12001.to_csv(output+'inner/inner_nio_era5_as12001.csv')

In [213]:
rlev_as22001=pd.DataFrame.from_dict(TCrl_ts[1])
prlev_as22001=pd.DataFrame.from_dict(TCpr_ts[1])
slev_as22001=pd.DataFrame.from_dict(TCsl_ts[1])
prlev_as22001['mslp']=prlev_as22001['mslp']/100

ts1_as22001=pd.concat([slev_as22001,rlev_as22001,prlev_as22001], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[1]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[1]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[1]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[1]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}

plev_as22001=pd.DataFrame.from_dict(alldict)

ts_as22001=pd.concat([ts1_as22001,plev_as22001], axis=1, join='inner')

ts_as22001

,u10,v10,2mdewtmp,2mtmp,conv_ppt,tot_cld_ice,tot_cldwtr,tot_cld_rain,vi_div_cld_froz_wtr,vi_div_cld_liq_wtr,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,1.592667,-5.081982,297.312263,300.172173,0.000321,0.150596,0.151042,0.302149,1.210370e-06,2.556597e-06,...,-0.331869,-0.313122,-0.235661,-0.158018,-0.134305,-0.213775,-0.219667,-0.140742,-0.061171,-0.022518
1,2.128597,-4.642845,297.418159,300.437813,0.000255,0.150944,0.135854,0.243529,1.567649e-06,2.565444e-06,...,-0.288792,-0.254566,-0.194470,-0.200366,-0.214222,-0.238308,-0.180450,-0.134362,-0.066663,-0.028060
2,2.337894,-4.213326,297.556725,300.483482,0.000409,0.170619,0.148261,0.168745,-2.897868e-07,-5.912734e-07,...,-0.290269,-0.294089,-0.233560,-0.134005,-0.085878,-0.110321,-0.111603,-0.067542,-0.019100,0.007075
3,1.964100,-3.787331,297.755707,300.556068,0.000221,0.165736,0.129022,0.122787,-4.951726e-07,-5.534384e-07,...,-0.281853,-0.252820,-0.109161,-0.044571,-0.053733,-0.113628,-0.107006,-0.056421,-0.019412,0.004264
4,0.659420,-2.583115,297.972336,300.426178,0.000441,0.167341,0.195340,0.142167,1.533190e-06,6.995397e-07,...,-0.301055,-0.265965,-0.201755,-0.263306,-0.313095,-0.285276,-0.166242,-0.104305,-0.057194,-0.023551
5,0.624973,-2.265770,297.896332,300.181259,0.001084,0.243986,0.207849,0.216759,1.877430e-06,1.266133e-06,...,-0.453619,-0.453167,-0.361646,-0.336571,-0.274140,-0.228187,-0.130778,-0.090525,-0.052924,-0.025261
6,-0.974979,-1.496605,297.502205,299.894875,0.000952,0.233545,0.150870,0.189075,1.840875e-06,1.821900e-08,...,-0.440048,-0.403472,-0.341825,-0.348852,-0.257944,-0.199222,-0.140443,-0.094048,-0.044036,-0.006734
7,-1.915878,-1.172585,297.577599,300.080292,0.000973,0.226519,0.157557,0.170859,2.414570e-06,-1.272419e-07,...,-0.390135,-0.365733,-0.369081,-0.351420,-0.207616,-0.134829,-0.078547,-0.062787,-0.028212,0.003287
8,-2.322186,-1.216610,297.637810,300.312111,0.000565,0.214117,0.130788,0.166910,2.103974e-06,1.076376e-06,...,-0.350999,-0.337510,-0.344680,-0.253571,-0.113941,-0.071643,-0.059719,-0.062746,-0.041344,-0.012799
9,-2.827938,-0.886864,297.629097,300.288475,0.000591,0.221511,0.114910,0.141728,1.663924e-06,-7.337658e-07,...,-0.347226,-0.349635,-0.327278,-0.210258,-0.113162,-0.088776,-0.056935,-0.051695,-0.042232,-0.027706


In [214]:
ts_as12001.to_csv(output+'inner/inner_nio_era5_as22001.csv')

In [167]:
rlev_bob22003=pd.DataFrame.from_dict(TCrl_ts[2])
prlev_bob22003=pd.DataFrame.from_dict(TCpr_ts[2])
slev_bob22003=pd.DataFrame.from_dict(TCsl_ts[2])
prlev_bob22003['mslp']=prlev_bob22003['mslp']/100

ts1_bob22003=pd.concat([slev_bob22003,rlev_bob22003,prlev_bob22003], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[2]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[2]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[2]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[2]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}

plev_bob22003=pd.DataFrame.from_dict(alldict)

ts_bob22003=pd.concat([ts1_bob22003,plev_bob22003], axis=1, join='inner')

ts_bob22003

,u10,v10,2mdewtmp,2mtmp,conv_ppt,tot_cld_ice,tot_cldwtr,tot_cld_rain,vi_div_cld_froz_wtr,vi_div_cld_liq_wtr,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,0.640701,1.528786,296.820877,299.153351,0.000590,0.190726,0.179403,0.200639,3.696885e-06,-1.294073e-06,...,-0.259044,-0.299023,-0.316048,-0.242935,-0.080206,-0.028415,-0.096155,-0.094598,-0.060228,-0.038049
1,2.866211,1.248283,296.781807,299.553215,0.000379,0.156705,0.172438,0.143039,3.412657e-06,2.334127e-08,...,-0.179273,-0.191629,-0.239891,-0.208583,-0.066313,-0.041783,-0.074574,-0.061424,-0.029991,-0.013072
2,2.492275,-0.400551,297.020500,299.389038,0.000520,0.147498,0.108188,0.128963,1.781271e-06,1.801411e-06,...,-0.171534,-0.162498,-0.221805,-0.150331,-0.056896,-0.115315,-0.087883,-0.046133,-0.008243,0.013371
3,2.475914,-0.792995,297.122368,299.428787,0.000805,0.112296,0.126034,0.132431,1.729059e-06,2.548506e-06,...,-0.164225,-0.118846,-0.180137,-0.204003,-0.194659,-0.261329,-0.152589,-0.090270,-0.042798,-0.009175
4,2.387924,-0.594162,297.136284,299.408958,0.000800,0.154113,0.194190,0.173135,3.889960e-06,3.241817e-06,...,-0.191979,-0.218875,-0.357105,-0.366105,-0.336426,-0.333528,-0.218360,-0.141328,-0.074604,-0.034559
5,1.993355,-0.990387,297.022453,299.407204,0.001072,0.195007,0.146537,0.176104,7.646158e-07,2.221321e-06,...,-0.309141,-0.340969,-0.370104,-0.318647,-0.228838,-0.149008,-0.095816,-0.082694,-0.051023,-0.024091
6,4.003380,-0.825661,296.782967,299.269821,0.000470,0.148003,0.111489,0.153949,1.188120e-06,1.868815e-06,...,-0.219716,-0.181474,-0.114429,-0.088005,-0.046167,-0.013978,0.012571,-0.014271,-0.002703,0.011372
7,1.249859,-1.976860,296.848915,299.655403,0.000429,0.176648,0.180554,0.287432,1.358666e-05,7.588882e-06,...,-0.242406,-0.252034,-0.265699,-0.332662,-0.376020,-0.360457,-0.222131,-0.159312,-0.070090,-0.019753
8,-0.662830,-2.113728,296.727165,299.524277,0.000645,0.241264,0.190265,0.383332,3.671797e-06,-1.866138e-07,...,-0.381906,-0.388693,-0.254512,-0.358204,-0.515158,-0.555194,-0.325670,-0.202633,-0.101058,-0.042554
9,-2.184353,-2.809074,296.800758,299.148384,0.001455,0.377000,0.202687,0.377268,3.234702e-06,1.422944e-06,...,-0.556253,-0.520959,-0.359207,-0.375499,-0.416988,-0.409403,-0.312689,-0.218272,-0.093832,-0.022489


In [168]:
ts_bob22003.to_csv(output+'inner/inner_nio_era5_bob22003.csv')

In [145]:
rlev_agni=pd.DataFrame.from_dict(TCrl_ts[3])
prlev_agni=pd.DataFrame.from_dict(TCpr_ts[3])
slev_agni=pd.DataFrame.from_dict(TCsl_ts[3])
prlev_agni['mslp']=prlev_agni['mslp']/100

ts1_agni=pd.concat([slev_agni,rlev_agni,prlev_agni], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[3]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[3]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[3]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[3]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}

plev_agni=pd.DataFrame.from_dict(alldict)

ts_agni=pd.concat([ts1_agni,plev_agni], axis=1, join='inner')

ts_agni

,u10,v10,2mdewtmp,2mtmp,conv_ppt,tot_cld_ice,tot_cldwtr,tot_cld_rain,vi_div_cld_froz_wtr,vi_div_cld_liq_wtr,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,-6.209557,-0.561646,296.776299,298.953545,0.001489,0.210344,0.123156,0.075017,5.315698e-06,1.589186e-06,...,-0.387722,-0.422020,-0.402140,-0.402594,-0.405169,-0.374021,-0.278668,-0.165313,-0.070502,-0.026807
1,-6.211090,-0.348244,296.740898,299.071350,0.001398,0.252471,0.111716,0.114945,6.574555e-06,2.582325e-06,...,-0.416982,-0.408790,-0.386669,-0.342566,-0.263473,-0.229460,-0.162078,-0.109574,-0.061780,-0.032999
2,-5.952991,-0.929688,296.755478,299.370514,0.000896,0.238151,0.112947,0.111895,1.370435e-05,3.421213e-06,...,-0.282278,-0.351051,-0.414619,-0.316819,-0.225535,-0.245809,-0.213694,-0.136020,-0.048665,0.000714
3,-4.492825,0.253735,296.932121,299.245468,0.001187,0.278084,0.105032,0.094500,1.026900e-05,3.808644e-06,...,-0.449879,-0.473701,-0.573090,-0.589495,-0.529046,-0.522026,-0.413724,-0.266906,-0.103667,-0.015755
4,-2.311817,0.881355,296.955673,299.160782,0.001280,0.519121,0.151974,0.199504,1.101347e-05,8.253846e-08,...,-0.705389,-0.694382,-0.686389,-0.621460,-0.487841,-0.439924,-0.362742,-0.237718,-0.096783,-0.028046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,-4.678654,-3.036427,295.518654,297.279427,0.000062,0.003674,0.118026,0.005280,-6.862683e-08,1.584180e-06,...,0.010048,0.036094,0.082416,0.087317,0.103942,0.103312,0.006471,-0.038877,-0.031301,-0.023615
63,-4.285057,-3.124840,295.395821,297.297943,0.000026,0.002173,0.089418,0.002694,5.218317e-08,1.939945e-06,...,-0.034555,0.004904,0.007879,0.012009,0.059685,0.085941,0.022418,-0.021171,-0.011384,-0.002151
64,-5.179806,-3.440529,295.077835,299.404930,0.000066,0.000269,0.090595,0.004278,2.299203e-08,1.200242e-06,...,-0.015046,0.064002,0.056567,0.036780,0.054286,0.059093,-0.026075,-0.046759,-0.036554,-0.030085
65,-5.125248,-2.720669,295.639069,300.428696,0.000059,0.000091,0.092147,0.008782,3.745663e-08,9.527430e-07,...,-0.034503,0.035449,0.062456,0.053165,0.005273,-0.041861,-0.106380,-0.082444,-0.051158,-0.041045


In [146]:
ts_agni.to_csv(output+'inner/inner_nio_era5_agni.csv')